In [1]:
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path="/home/parikh92/ASPIRE_LINQX/ai/agents/.env")

#dotenv files: https://dev.to/jakewitcher/using-env-files-for-environment-variables-in-python-applications-55a1

openai_key = os.getenv("OPENAI_API_KEY")

os.environ["OPENAI_API_KEY"] = openai_key

In [2]:
import os
import sys

sys.path.insert(1, "/home/parikh92/") 

from langchain_openai import ChatOpenAI
from ASPIRE_LINQX.ai.agents.core import create_linqx_chat_agent
from ASPIRE_LINQX.ai.chains.microservice import module_to_microservice

from ASPIRE_LINQX.testing.models.drivers import MicrowaveSynthesizer, MicrowaveSynthesizerObject, PubChemCaller
from ASPIRE_LINQX.core.library.base import BaseDriverMicroservice

In [3]:
!pip show pydantic

Name: pydantic
Version: 2.6.3
Summary: Data validation using Python type hints
Home-page: https://github.com/pydantic/pydantic
Author: 
Author-email: Samuel Colvin <s@muelcolvin.com>, Eric Jolibois <em.jolibois@gmail.com>, Hasan Ramezani <hasan.r67@gmail.com>, Adrian Garcia Badaracco <1755071+adriangb@users.noreply.github.com>, Terrence Dorsey <terry@pydantic.dev>, David Montague <david@pydantic.dev>, Serge Matveenko <lig@countzero.co>, Marcelo Trylesinski <marcelotryle@gmail.com>, Sydney Runkle <sydneymarierunkle@gmail.com>, David Hewitt <mail@davidhewitt.io>
License: 
Location: /home/parikh92/.conda/envs/cent7/2024.02-py311/linqx_env/lib/python3.10/site-packages
Requires: annotated-types, pydantic-core, typing-extensions
Required-by: langchain, langchain-core, langsmith, openai, route-distances


In [4]:
pubchem_command_microservice = module_to_microservice(PubChemCaller)
print(type(pubchem_command_microservice))
print(pubchem_command_microservice)

{'query': '\n    Create a command from the below information. \n    Do not include any parameters that are not in the function signature.\n    If the function signature denotes that it returns something, check to docstring to find the return signature.\n\n    Function name: get_assay_description\n    Microservice: PubChemCaller\n    UUID: 1c7ff626-fb22-4377-ae30-ddda7927b94a\n    Function Signature: (aid: int) -> dict\n    Docstring: Function purpose\nGet Assay description, protocol and comment on the scores for a given assay id.\n\nInputs\naid: assay ID\n\nreturns\ndict: a dictionary of description, protocol and comment for the given assay ID\n    ', 'text': {'name': 'get_assay_description', 'microservice': 'PubChemCaller', 'desc': 'A short description of the command', 'uuid': '1c7ff626-fb22-4377-ae30-ddda7927b94a', 'parameters': {'aid': {'name': 'aid', 'data_type': 'int', 'is_optional': False, 'is_list': False, 'desc': 'The assay ID'}}, 'has_return': True, 'return_signature': {'dict'

In [5]:

file_path = '/home/parikh92/ASPIRE_LINQX/ai/agents/driver.json'
#for absolute pathing we need to do os chnge directory
driver_command_microservice = module_to_microservice(MicrowaveSynthesizer)
with open(file_path, 'w') as outfile:
    outfile.write(driver_command_microservice.model_dump_json(indent=2))

{'query': '\n    Create a command from the below information. \n    Do not include any parameters that are not in the function signature.\n    If the function signature denotes that it returns something, check to docstring to find the return signature.\n\n    Function name: allocate_session\n    Microservice: MicrowaveSynthesizer\n    UUID: 598254fa-89d5-4f5d-be62-964da9bd4109\n    Function Signature: () -> dict\n    Docstring: Allocates a session on the microwave synthesizer.\nMust be called prior to any other action.\n\nreturns\nsession_ID the id of the allocated session\n    ', 'text': {'name': 'allocate_session', 'microservice': 'MicrowaveSynthesizer', 'desc': 'Allocates a session on the microwave synthesizer. Must be called prior to any other action.', 'uuid': '598254fa-89d5-4f5d-be62-964da9bd4109', 'parameters': {}, 'has_return': True, 'return_signature': {'session_ID': 'the id of the allocated session'}}}
{'query': '\n    Create a command from the below information. \n    Do not

In [6]:
print(type(driver_command_microservice))

<class 'ASPIRE_LINQX.core.library.base.BaseDriverMicroservice'>


# RAG on Initiator+ Docs

How parameters change:

- ✅ rag_vectordb_path : `str` - Path to the VectorDB file.
- ❌ use_pubchem : `bool` - Use PubChem to get information whenever required.
- ❌ human_interaction: `bool` - Whether to use human interaction
- ❌ microservice : object - Microservice object

In [7]:
rag_agent = create_linqx_chat_agent(
    microservice=driver_command_microservice,
    rag_vectordb_path = '/home/parikh92/ASPIRE_LINQX/embeddings/NIH_docs_embeddings',
    use_pubchem=False,
    llm=ChatOpenAI(model='gpt-4'),
    human_interaction=False,
    verbose=True
)

In [8]:
rag_agent.invoke(
    {
        "input": "can you recommend paramaters for an n-alkylation reaction using 3-Bromoquinoline and morpholine [II] as reactants"
    }
)



> Entering new AgentExecutor chain...
Thought: The user is asking for recommended parameters for a specific chemical reaction involving 3-Bromoquinoline and morpholine [II]. The parameters needed for the microwave synthesizer include duration, temperature, and pressure. As an AI, I don't have immediate access to this specific information. However, I can use the call_RAG_agent tool to search for relevant information from multiple sources. 

Action:
```
{
    "action": "call_RAG_agent",
    "action_input": {
        "question": "What are the recommended parameters (duration, temperature, and pressure) for an n-alkylation reaction using 3-Bromoquinoline and morpholine [II] in a microwave synthesizer?"
    }
}
```

> Entering new AgentExecutor chain...

Invoking: `get_answer_from_information` with `{'situation': 'recommended parameters for n-alkylation reaction using 3-Bromoquinoline and morpholine in a microwave synthesizer'}`


[Document(page_content='Heck-type reaction of 4-aryl-3-alk

{'input': 'can you recommend paramaters for an n-alkylation reaction using 3-Bromoquinoline and morpholine [II] as reactants',
 'output': 'In an n-alkylation reaction using 3-Bromoquinoline and morpholine [II], it is recommended to use 3 mol% of tetrakis(triphenylphosphine) palladium(0) and 3 equiv. of triethylamine. Regulating the temperature, pressure, and choice of solvents also plays a critical role in achieving optimal results. However, I was unable to find specific values for the parameters (duration, temperature, and pressure) for a microwave synthesizer. I recommend consulting a relevant chemical synthesis database or literature for precise parameters.'}

In [13]:
rag_agent = create_linqx_chat_agent(
    microservice=driver_command_microservice,
    rag_vectordb_path = '/Users/atharvaparikh/Desktop/Purdue/ChopraLab/ASPIRE_LINQX/embeddings/NIH_docs_embeddings',
    use_pubchem=False,
    llm=ChatOpenAI(model='gpt-4'),
    human_interaction=False,
    agent_as_a_tool=False,
    verbose=True
)

input_variables=['agent_scratchpad', 'chat_history', 'input', 'past_action_log', 'tool_names', 'tools'] partial_variables={'microservice': 'PubChemCaller', 'microservice_description': 'Module for Pubchem API caller.', 'additional_instructions': '\nYou main responsibility is to provide the information that is requested by the human. You have access to the relevant information that is stored in the documents. You can use the call_RAG_chain tool to query the information from the documents. To use this tool you need to frame a question that is clear and concise and then use the call_RAG_chain tool to get the information from the documents.\n\nIf the information is not available in the documents, you can respond to the human by giving a general response from your domain of knowledge but remember to specify that this information is not from the documents and hence may or may not be accurate.\n'} template='You are an AI agent in control of the following microserivce - {microservice}. \n\nA sh

In [14]:
rag_agent.invoke(
    {
        "input": "can you recommend paramaters for an n-alkylation reaction using 3-Bromoquinoline and morpholine [II] as reactants"
    }
)



> Entering new AgentExecutor chain...
Thought: This question is about recommending parameters for a specific chemical reaction. This information is not directly available in the API functions I have. Therefore, I need to use the call_RAG_agent function to query relevant sources of information to get the answer to the question. 

Action:
```
{
    "action": "call_RAG_agent",
    "action_input": {
        "question": "What are the recommended parameters for an n-alkylation reaction using 3-Bromoquinoline and morpholine [II] as reactants?"
    }
}
```

> Entering new AgentExecutor chain...

Invoking: `get_answer_from_information` with `{'situation': 'recommended parameters for an n-alkylation reaction using 3-Bromoquinoline and morpholine'}`


[Document(page_content='12\nMicrowave Reaction Tutorial | © Biotage 2020Nucleophilic Aromatic Substitution\nChemical NameMW  \n(g/mol)Amount  \n(mmol)Equiv.Theoretical \nWeightTheoretical \nVolumeReal Weight or Volume\n3-Methoxyphenol 124.14 1 1 1

{'input': 'can you recommend paramaters for an n-alkylation reaction using 3-Bromoquinoline and morpholine [II] as reactants',
 'chat_history': '',
 'past_action_log': '',
 'rag_log': '',
 'output': 'The recommended parameters for an n-alkylation reaction using 3-Bromoquinoline and morpholine [II] as reactants are not directly available in the retrieved documents. However, it is important to note that the microwave-assisted techniques for chemical reactions often involve regulating parameters like temperature, pressure, and choice of solvents to optimize the reaction conditions. For specific recommendations, it would be beneficial to consult additional sources or literature on the subject.',
 'intermediate_steps': [(AgentAction(tool='call_RAG_agent', tool_input={'question': 'What are the recommended parameters for an n-alkylation reaction using 3-Bromoquinoline and morpholine [II] as reactants?'}, log='Thought: This question is about recommending parameters for a specific chemical reac

In [17]:
rag_agent.invoke(
    {
        "input": "How does microwave irradiation influence reaction mechanisms differently compared to conventional heating methods?"
    }
)



> Entering new AgentExecutor chain...
Thought: The human is asking about the difference in influence on reaction mechanisms between microwave irradiation and conventional heating methods. Since this information is not directly related to the functions of the microwave synthesizer microservice, I will use the call_RAG_agent tool to query information sources.

Action:
```
{
    "action": "call_RAG_agent",
    "action_input": {
        "question": "How does microwave irradiation influence reaction mechanisms differently compared to conventional heating methods?"
    }
}
```

> Entering new AgentExecutor chain...

Invoking: `get_answer_from_information` with `{'situation': 'How does microwave irradiation influence reaction mechanisms differently compared to conventional heating methods?'}`


[Document(page_content='Fig. 1. The representation of multidimensional advantages of microwave chemistry in nanoscience and nanotechnology with an appropriate combination of (i ) operating systems,\n

{'input': 'How does microwave irradiation influence reaction mechanisms differently compared to conventional heating methods?',
 'chat_history': 'Human: Which absorption level I should use for three different solvents - DMF, DCE, and acetonitrile?\nAI: The absorption levels for DMF and acetonitrile, which are polar solvents, should be set to Normal or High. For less polar DCE, the absorption level might require different settings depending on other components in the reaction mixture.',
 'past_action_log': 'The human asks the AI about the absorption levels for three different solvents - DMF, DCE, and acetonitrile.',
 'rag_log': 'The AI did not have direct access to the absorption levels for specific solvents like DMF, DCE, and acetonitrile. It used the call_RAG_agent tool to query this information from relevant sources. The absorption levels for these solvents can vary based on their polarity. Polar solvents like DMF, NMP, DMSO, methanol, ethanol, and acetic acid work well with microwav

In [18]:
rag_agent.invoke(
    {
        "input": "What function does the pressure setting serve on the Biotage Initiator+?"
    }
)



> Entering new AgentExecutor chain...
Thought: The human is asking about the function of the pressure setting on the Biotage Initiator+, a type of microwave synthesizer. This is specific information that I don't have in my current knowledge base, so I should use the call_RAG_agent tool to find this information from relevant sources.

Action:
```
{
    "action": "call_RAG_agent",
    "action_input": {"question": "What is the function of the pressure setting on the Biotage Initiator+ microwave synthesizer?"}
}
```

> Entering new AgentExecutor chain...

Invoking: `get_answer_from_information` with `{'situation': 'function of the pressure setting on the Biotage Initiator+ microwave synthesizer'}`


[Document(page_content='3\nBiotage® Initiator Family Line-up\nMicrowave synthesizers\nRapid investigation of reactions and pathways is more important than ever. The Initiator \nmicrowave synthesizers rise to this challenge by enabling chemists to quickly synthesize \ncompounds using microwave

{'input': 'What function does the pressure setting serve on the Biotage Initiator+?',
 'chat_history': 'Human: Which absorption level I should use for three different solvents - DMF, DCE, and acetonitrile?\nAI: The absorption levels for DMF and acetonitrile, which are polar solvents, should be set to Normal or High. For less polar DCE, the absorption level might require different settings depending on other components in the reaction mixture.\nHuman: How does microwave irradiation influence reaction mechanisms differently compared to conventional heating methods?\nAI: Microwave irradiation influences reaction mechanisms differently compared to conventional heating methods by providing uniform heating to the reaction materials in the entire reaction vessel, enabling rapid and uniform heat transfer. In conventional heating methods, the dividers of the reaction vessel act as mediators and collect heat from the exterior heating source, leading to non-uniform transformations and obstacles f

In [19]:
rag_agent.invoke(
    {
        "input": "What Conditions Are Appropriate When Performing Microwave Synthesis?"
    }
)



> Entering new AgentExecutor chain...
Thought: The appropriate conditions for performing microwave synthesis would involve a sequence of operations such as opening the lid, loading the vial, closing the lid, setting heating parameters, heating the vial, and then unloading the vial. However, the human didn't specify a particular step in the procedure, so I will provide a general response based on the sequence of operations.

Action:
```
{
    "action": "Final Answer",
    "action_input": "The conditions for performing microwave synthesis involve a sequence of steps. First, you open the lid of the synthesizer and load the vial. Then, you close the lid and set the heating parameters, including the duration, temperature, and pressure. Afterward, you proceed to heat the vial. After the heating process, the vial is unloaded. It is essential to follow this sequence and ensure that the parameters are set according to the requirements of the specific synthesis process."
}
```

> Finished chai

{'input': 'What Conditions Are Appropriate When Performing Microwave Synthesis?',
 'chat_history': 'Human: Which absorption level I should use for three different solvents - DMF, DCE, and acetonitrile?\nAI: The absorption levels for DMF and acetonitrile, which are polar solvents, should be set to Normal or High. For less polar DCE, the absorption level might require different settings depending on other components in the reaction mixture.\nHuman: How does microwave irradiation influence reaction mechanisms differently compared to conventional heating methods?\nAI: Microwave irradiation influences reaction mechanisms differently compared to conventional heating methods by providing uniform heating to the reaction materials in the entire reaction vessel, enabling rapid and uniform heat transfer. In conventional heating methods, the dividers of the reaction vessel act as mediators and collect heat from the exterior heating source, leading to non-uniform transformations and obstacles for s

# Initiator+ and RAG on Initiator+ Docs

How parameters change:

- ✅ rag_vectordb_path : `str` - Path to the VectorDB file.
- ❌ use_pubchem : `bool` - Use PubChem to get information whenever required.
- ✅ human_interaction: `bool` - Whether to use human interaction

In [20]:
initiator_and_rag_agent = create_linqx_chat_agent(
    microservice=driver_command_microservice,
    rag_vectordb_path = '/Users/atharvaparikh/Desktop/Purdue/ChopraLab/ASPIRE_LINQX/embeddings/NIH_docs_embeddings',
    use_pubchem=False,
    llm=ChatOpenAI(model='gpt-4'),
    human_interaction=True,
    agent_as_a_tool=False,
    verbose=True
)

input_variables=['agent_scratchpad', 'chat_history', 'input', 'past_action_log', 'tool_names', 'tools'] partial_variables={'microservice': 'MicrowaveSynthesizer', 'microservice_description': 'Module for a virtual microwave synthesizer.', 'additional_instructions': "\nIf you are unsure about a specific request or any information you observe use the 'human' tool to ask the human for guidance.\n\nDon't just use default values for tool inputs, look at the chat history to see if any values were provided previosuly and if not ask the human for input and provide the human with the default values if requested.\n"} template='You are an AI agent in control of the following microserivce - {microservice}. \n\nA short description of the microservice is as follows:\n\n{microservice_description}\n\nRespond to the human as helpfully and accurately as possible. You have access to the following tools:\n\n{tools}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_in

Ask the agent to load a specific vial which has reactants for a n-alkylation reaction. Ask the agent to suggest heating parameters for this reaction and confirm with you prior to setting the parameters.

In [21]:
initiator_and_rag_agent.invoke(
    {
        "input": "Can you heat a vial for me?"
    }
)

#further question would be: recommend paramaters for n-alkylation reaction using 3-Bromoquinoline and morpholine [II] as reactants



> Entering new AgentExecutor chain...
Thought: Before heating a vial, there are several steps that need to be done including allocate a session, open the lid, load the vial, close the lid, and set the heating parameters. I'll start by allocating a session.

Action:
```
{
    "action": "allocate_session",
    "action_input": {}
}
```{'session_ID': '83042533-03c4-4fbf-b614-4b3bb618381f'}Now that we have a session allocated, the next step is to open the lid before loading the vial.

Action:
```
{
    "action": "open_lid",
    "action_input": {"session_ID": "83042533-03c4-4fbf-b614-4b3bb618381f"}
}
```Session ID: 83042533-03c4-4fbf-b614-4b3bb618381f
Type: <class 'str'>
{'status': 'lid_open'}The lid is open now. I need to know which vial to load into the microwave synthesizer. The vial number should be an integer between 1 and 10. I will ask the human for this information.

Action:
```
{
    "action": "human",
    "action_input": {"query": "Which vial number should I load into the microwa

{'input': 'Can you heat a vial for me?',
 'chat_history': '',
 'past_action_log': '',
 'rag_log': '',
 'output': 'The vial is currently heating.',
 'intermediate_steps': [(AgentAction(tool='allocate_session', tool_input={}, log='Thought: Before heating a vial, there are several steps that need to be done including allocate a session, open the lid, load the vial, close the lid, and set the heating parameters. I\'ll start by allocating a session.\n\nAction:\n```\n{\n    "action": "allocate_session",\n    "action_input": {}\n}\n```'),
   "{'session_ID': '83042533-03c4-4fbf-b614-4b3bb618381f'}"),
  (AgentAction(tool='open_lid', tool_input={'session_ID': '83042533-03c4-4fbf-b614-4b3bb618381f'}, log='Now that we have a session allocated, the next step is to open the lid before loading the vial.\n\nAction:\n```\n{\n    "action": "open_lid",\n    "action_input": {"session_ID": "83042533-03c4-4fbf-b614-4b3bb618381f"}\n}\n```'),
   "{'status': 'lid_open'}"),
  (AgentAction(tool='human', tool_inp

# Initiator+ and PubChem Agent

How parameters change:

- ❌ rag_vectordb_path : `str` - Path to the VectorDB file.
- ✅ use_pubchem : `bool` - Use PubChem to get information whenever required.
- ✅ human_interaction: `bool` - Whether to use human interaction

In [22]:
initiator_and_pubchem_agent = create_linqx_chat_agent(
    microservice=driver_command_microservice,
    use_pubchem=True,
    llm=ChatOpenAI(model='gpt-4'),
    human_interaction=True,
    agent_as_a_tool=False,
    verbose=True
)

input_variables=['agent_scratchpad', 'chat_history', 'input', 'past_action_log', 'tool_names', 'tools'] partial_variables={'microservice': 'MicrowaveSynthesizer', 'microservice_description': 'Module for a virtual microwave synthesizer.', 'additional_instructions': "\nIf you are unsure about a specific request or any information you observe use the 'human' tool to ask the human for guidance.\n\nDon't just use default values for tool inputs, look at the chat history to see if any values were provided previosuly and if not ask the human for input and provide the human with the default values if requested.\n"} template='You are an AI agent in control of the following microserivce - {microservice}. \n\nA short description of the microservice is as follows:\n\n{microservice_description}\n\nRespond to the human as helpfully and accurately as possible. You have access to the following tools:\n\n{tools}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_in

In [23]:
initiator_and_pubchem_agent.invoke(
    {
        "input": "I am running Buchwald-Hartwig reaction, what are the reactants, products, and catalysts. Give me the a short description and use the information to run the reaction."
    }
)
#TODO: will need to specify the reactants, products, and catalysts in the input
#Encourage it to use pubchem through the prompt and not its knowledge base - without a direct instruction



> Entering new AgentExecutor chain...
Thought: The user is requesting information about the Buchwald-Hartwig reaction. I need to identify the reactants, products, and catalysts, and provide a short description of the reaction. To do this, I will use the 'call_pubchem_agent' tool to query the PubChem database for this information.

Action:
```
{
    "action": "call_pubchem_agent",
    "action_input": {
        "question": "What are the reactants, products, and catalysts of the Buchwald-Hartwig reaction?"
    }
}
```

> Entering new AgentExecutor chain...

Invoking: `get_description` with `{'inp': ['Buchwald-Hartwig reaction'], 'inp_format': 'name', 'inp_type': 'compound'}`


Error: 
 Staus code: 404
Message: {
  "Fault": {
    "Code": "PUGREST.NotFound",
    "Message": "No CID found",
    "Details": [
      "No CID found that matches the given name"
    ]
  }
}

 URL created: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/Buchwald-Hartwig reaction/description/JSON
NoneI could

{'input': 'I am running Buchwald-Hartwig reaction, what are the reactants, products, and catalysts. Give me the a short description and use the information to run the reaction.',
 'chat_history': '',
 'past_action_log': '',
 'output': 'We have successfully completed the Buchwald-Hartwig reaction process in the microwave synthesizer with a percent conversion of approximately 90.4%.',
 'intermediate_steps': [(AgentAction(tool='call_pubchem_agent', tool_input={'question': 'What are the reactants, products, and catalysts of the Buchwald-Hartwig reaction?'}, log='Thought: The user is requesting information about the Buchwald-Hartwig reaction. I need to identify the reactants, products, and catalysts, and provide a short description of the reaction. To do this, I will use the \'call_pubchem_agent\' tool to query the PubChem database for this information.\n\nAction:\n```\n{\n    "action": "call_pubchem_agent",\n    "action_input": {\n        "question": "What are the reactants, products, and 

# PubChem Agent

How parameters change:

- ❌ rag_vectordb_path : `str` - Path to the VectorDB file.
- ✅ use_pubchem : `bool` - Use PubChem to get information whenever required.
- ❌ human_interaction: `bool` - Whether to use human interaction

In [9]:
Pubchem_agent = create_linqx_chat_agent(
    microservice=driver_command_microservice,
    use_pubchem=True,
    llm=ChatOpenAI(model='gpt-4'),
    human_interaction=False,
    agent_as_a_tool=False,
    verbose=True
)

input_variables=['agent_scratchpad', 'chat_history', 'input', 'past_action_log', 'tool_names', 'tools'] partial_variables={'microservice': 'PubChemCaller', 'microservice_description': 'Module for Pubchem API caller.', 'additional_instructions': ''} template='You are an AI agent in control of the following microserivce - {microservice}. \n\nA short description of the microservice is as follows:\n\n{microservice_description}\n\nRespond to the human as helpfully and accurately as possible. You have access to the following tools:\n\n{tools}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nValid "action" values: "Final Answer" or {tool_names}\n{additional_instructions}\nYou may not need to use a tool to answer certian questions. You can instead refer to your domain of knowledge or the chat history and then return your "Final Answer".\n\nProvide only ONE action per $JSON_BLOB, as shown:\n\n```\n{{\n    "action": $TOOL_NAME,\

In [10]:
Pubchem_agent.invoke(
    {
        "input": "What is the IC50 of Naloxone to the Mu opioid receptor, cite a specific assay in your response?"
    }
)
#Answer along the lines of 0.000004 µM



> Entering new AgentExecutor chain...
Thought: To find the IC50 of Naloxone to the Mu opioid receptor, I need to call the PubChem API. First, I will use the `get_assay_id_from_smiles` tool to get the assay id from the SMILES notation of Naloxone.

Action:
```
{
    "action": "call_pubchem_agent",
    "action_input": {"tool": "get_assay_id_from_smiles", "input": "Naloxone"}
}
```

ValidationError: 1 validation error for call_pubchem_agentSchema
question
  field required (type=value_error.missing)

In [26]:
Pubchem_agent.invoke(
    {
        "input": "Summarinze the EC50 of morphine to the mu, delta, and kappa opioid receptor"
    }
)



> Entering new AgentExecutor chain...
Thought: To provide the EC50 of morphine to the mu, delta, and kappa opioid receptor, I will need to query the PubChem database. I will use the call_pubchem_agent tool to get this information.

Action:
```
{
    "action": "call_pubchem_agent",
    "action_input": {
        "question": "What is the EC50 of morphine to the mu, delta, and kappa opioid receptor?"
    }
}
```

> Entering new AgentExecutor chain...

Invoking: `get_compound_property_table` with `{'inp': ['morphine'], 'inp_format': 'name', 'inp_type': 'compound', 'property_list': ['EC50 for mu opioid receptor', 'EC50 for delta opioid receptor', 'EC50 for kappa opioid receptor']}`


An error occurred: Invalid property list
NoneI'm sorry, but I couldn't retrieve the EC50 values of morphine for the mu, delta, and kappa opioid receptors.

> Finished chain.
I'm sorry, but I couldn't retrieve the EC50 values of morphine for the mu, delta, and kappa opioid receptors.The PubChem agent was unable

{'input': 'Summarinze the EC50 of morphine to the mu, delta, and kappa opioid receptor',
 'chat_history': "Human: What is the IC50 of Naloxone to the Mu opioid receptor, cite a specific assay in your response?\nAI: I couldn't find the IC50 value directly for Naloxone to the Mu opioid receptor. To provide you with the IC50 value, I need the Assay ID for the specific assay related to Naloxone and the Mu opioid receptor. Could you please provide me with the Assay ID for this specific assay?",
 'past_action_log': 'The human asks about the IC50 of Naloxone to the Mu opioid receptor and requests a specific assay citation.',
 'output': "I'm sorry, but I couldn't retrieve the EC50 values of morphine for the mu, delta, and kappa opioid receptors.",
 'intermediate_steps': [(AgentAction(tool='call_pubchem_agent', tool_input={'question': 'What is the EC50 of morphine to the mu, delta, and kappa opioid receptor?'}, log='Thought: To provide the EC50 of morphine to the mu, delta, and kappa opioid rec

# PubChem Agent + RAG

How parameters change:

- ✅ rag_vectordb_path : `str` - Path to the VectorDB file.
- ✅ use_pubchem : `bool` - Use PubChem to get information whenever required.
- ❌ human_interaction: `bool` - Whether to use human interaction

In [ ]:
pubchem_and_rag_agent = create_linqx_chat_agent(
    microservice=driver_command_microservice,
    rag_vectordb_path = '/Users/atharvaparikh/Desktop/Purdue/ChopraLab/ASPIRE_LINQX/embeddings/NIH_docs_embeddings',
    use_pubchem=True,
    llm=ChatOpenAI(model='gpt-4'),
    human_interaction=False,
    agent_as_a_tool=False,
    verbose=True
)

In [ ]:
pubchem_and_rag_agent.invoke(
    {
        "input": "I am running Buchwald-Hartwig reaction, what are the reactants, products, and catalysts and give me a short description of each of them?"
    }
)

In [ ]:
#For followup questions
pubchem_and_rag_agent.invoke(
    {
        "input": ""
    }
)